# Projet maison n° 3

In [1]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    from glob import glob
    import os
    
    pth = "**/names/*.txt"
    csv_paths = glob(pth, recursive=True)
    
    df = pd.DataFrame(columns=['year', 'name', 'gender', 'births'])
    
    for csv_path in csv_paths:
        df1 = pd.read_csv(csv_path, names = ['name', 'gender', 'births'])
        df1.insert(0, 'year', os.path.basename(csv_path)[3:7])
        df = pd.concat([df, df1])
    df = df.sort_values(by=['year', 'gender', 'births'], ascending=[True, True, False]).reset_index(drop=True)
    
    return df

## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [3]:
df = pd.read_csv('nat2020.csv', sep=';', dtype=object)
df.loc[df['preusuel'].isnull()]

,sexe,preusuel,annais,nombre
563597,2,NaN,2003,3
563598,2,NaN,XXXX,29


In [4]:
# names fr
def df_names_fr():
    df = pd.read_csv('nat2020.csv', sep=';', dtype=object, na_filter=False)
    df = df.rename(columns={'sexe': 'gender', 'preusuel': 'name', 'annais':'year', 'nombre': 'births'})
    df = df[['year', 'name', 'gender', 'births']]
    df['gender'] = df['gender'].map({'1':'M', '2':'F'})
    df = df[df['name'].astype(str).apply(len) >= 2]
    df = df.drop(df.loc[df['year'] == 'XXXX'].index)
    df = df.drop(df.loc[df['name'] == '_PRENOMS_RARES'].index)
    df['name'] = df['name'].astype(str).str.capitalize()
    df['births'] = df['births'].astype(int)
    df = df.sort_values(by=['year', 'gender', 'births', 'name'], ascending=[True, True, False, True])
    df['births'] = df['births'].astype(object)
    df.index = pd.RangeIndex(stop=len(df.index))
    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [5]:
# taux de change
def df_taux_change(devises):
    df = pd.read_csv('Webstat_Export_20210929.csv', sep=';', skiprows=range(1,6), index_col='Titre :', na_values=['-'])
    df.columns = df.columns.map(lambda x : x[-4:-1])
    df.index.name = None
    df = df.dropna()
    df.index = pd.to_datetime(df.index, format='%d/%m/%Y')
    df = df.apply(lambda x: x.str.replace(',','.'))
    df = df.astype('float')
    df = pd.DataFrame(df, columns=devises)
    return df

### Tests

In [6]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630407)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [7]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [8]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 11.423s

OK
